# Data Wrangling with DataFrames Coding Quiz

Use this Jupyter notebook to find the answers to the quiz in the previous section. There is an answer key in the next part of the lesson.

In [30]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, desc, asc, col
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType
# TODOS: 
# 1) import any other libraries you might need
# 2) instantiate a Spark session 
# 3) read in the data set located at the path "data/sparkify_log_small.json"
# 4) write code to answer the quiz questions 

In [2]:
spark = SparkSession \
.builder \
.appName("Wrangling Data") \
.getOrCreate()

In [3]:
path = "data/sparkify_log_small.json"
user_log = spark.read.json(path)

# Question 1

Which page did user id "" (empty string) NOT visit?

In [4]:
user_log.take(5)

[Row(artist='Showaddywaddy', auth='Logged In', firstName='Kenneth', gender='M', itemInSession=112, lastName='Matthews', length=232.93342, level='paid', location='Charlotte-Concord-Gastonia, NC-SC', method='PUT', page='NextSong', registration=1509380319284, sessionId=5132, song='Christmas Tears Will Fall', status=200, ts=1513720872284, userAgent='"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"', userId='1046'),
 Row(artist='Lily Allen', auth='Logged In', firstName='Elizabeth', gender='F', itemInSession=7, lastName='Chase', length=195.23873, level='free', location='Shreveport-Bossier City, LA', method='PUT', page='NextSong', registration=1512718541284, sessionId=5027, song='Cheryl Tweedy', status=200, ts=1513720878284, userAgent='"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"', userId='1000'),
 Row(artist='Cobra Starship Featuring Leighton Meester', auth='Logged In', firstNa

In [5]:
user_log.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [6]:
user_log.select("page").dropDuplicates().sort("page").show()

+----------------+
|            page|
+----------------+
|           About|
|       Downgrade|
|           Error|
|            Help|
|            Home|
|           Login|
|          Logout|
|        NextSong|
|   Save Settings|
|        Settings|
|Submit Downgrade|
|  Submit Upgrade|
|         Upgrade|
+----------------+



In [7]:
user_log.select(["userId", "page"]).dropDuplicates().sort("page").where(user_log.userId =='').collect()

[Row(userId='', page='About'),
 Row(userId='', page='Help'),
 Row(userId='', page='Home'),
 Row(userId='', page='Login')]

In [ ]:
# TODO: write your code to answer question 1

# Question 2 - Reflect

What type of user does the empty string user id most likely refer to?


#use this space to explore the behavior of the user with an empty string
Users who are only visiting the About, Home and Login pages are probably unregistered visitors.


# Question 3

How many female users do we have in the data set?

In [8]:
# TODO: write your code to answer question 3
user_log.select("gender").dropDuplicates().sort("gender").show()

+------+
|gender|
+------+
|  null|
|     F|
|     M|
+------+



In [12]:
user_log.dropDuplicates(["userId"]).groupby(user_log.gender).count().show()

+------+-----+
|gender|count|
+------+-----+
|     F|  462|
|  null|    1|
|     M|  501|
+------+-----+



In [19]:
user_log.filter(user_log.gender == 'F') \
    .select('userId', 'gender') \
    .dropDuplicates() \
    .count()

462

# Question 4

How many songs were played from the most played artist?

In [23]:
# TODO: write your code to answer question 4
user_log.dropna(how='any', subset='artist').groupby([user_log.artist]).count().orderBy('count', ascending=False).show(1)

+--------+-----+
|  artist|count|
+--------+-----+
|Coldplay|   83|
+--------+-----+
only showing top 1 row



In [20]:
user_log.filter(user_log.page == 'NextSong') \
    .select('Artist') \
    .groupBy('Artist') \
    .agg({'Artist':'count'}) \
    .withColumnRenamed('count(Artist)', 'Artistcount') \
    .sort(desc('Artistcount')) \
    .show(1)

+--------+-----------+
|  Artist|Artistcount|
+--------+-----------+
|Coldplay|         83|
+--------+-----------+
only showing top 1 row



# Question 5 (challenge)

How many songs do users listen to on average between visiting our home page? Please round your answer to the closest integer.



In [ ]:
# TODO: write your code to answer question 5

In [26]:
function = udf(lambda ishome : int(ishome =='Home'), IntegerType())

user_window = Window \
.partitionBy('userId') \
.orderBy(desc('ts')) \
.rangeBetween(Window.unboundedPreceding, 0)

In [31]:
cusum = user_log.filter((user_log.page == 'NextSong') | (user_log.page == 'Home')) \
.select('userId', 'page', 'ts')\
.withColumn('homevisit', function(col('page'))) \
.withColumn('period', Fsum('homevisit').over(user_window)) 


DataFrame[userId: string, page: string, ts: bigint, homevisit: int, period: bigint]

In [32]:
cusum.take(1)

[Row(userId='1436', page='NextSong', ts=1513783259284, homevisit=0, period=0)]

In [33]:
cusum.filter((cusum.page =='NextSong')) \
.groupby('userID', 'period')\
.agg({'period':'count'})\
.agg({'count(period)':'avg'}).show()

+------------------+
|avg(count(period))|
+------------------+
| 6.898347107438017|
+------------------+

